### BIOSTAT 257: HW3

#### Q1: Formula

In [1]:
# load libraries
using BenchmarkTools, DelimitedFiles, Images, LinearAlgebra, Random

In [ ]:
# write down the form of the log-likelihood of the above mixed model. written on ipad, copy down later

#### Q1: Formula

Consider a linear mixed effects model,
        $$\mathbf{Y}_i = \mathbf{X}_i \boldsymbol{\beta} + \mathbf{Z}_i \boldsymbol{\gamma} + \boldsymbol{\epsilon}_i, \quad i=1,\ldots,n $$
 where,
 - $\mathbf{Y}_i \in \mathbb{R}^{n_i}$ is the response vector of $i$-th individual,
 - $\mathbf{X}_i \in \mathbb{R}^{n_i \times p}$ is the fixed effect predictor matrix of $i$-th individual,
 - $\mathbf{Z}_i \in \mathbb{R}^{n_i \times q}$ is the random effect predictor matrix of $i$-th individuaL,
 - $\boldsymbol{\epsilon}_i \in \mathbb{R}^{n_i}$ are multivariate normal $N(\mathbf{0}_{n_i},\sigma^2 \mathbf{I}_{n_i})$, 
 - $\boldsymbol{\beta} \in \mathbb{R}^p$ are fixed effects, and
 - $\boldsymbol{\gamma} \in \mathbb{R}^q$ are random effects assumed to be $N(\mathbf{0}_q, \boldsymbol{\Sigma}_{q \times q}$) independent of $\boldsymbol{\epsilon}_i$.

#### Q2: Start-up code

Use the following template to define a type LmmObs that holds an LMM datum  (𝐲𝑖,𝐗𝑖,𝐙𝑖) .

In [2]:
struct LmmObs{T <: AbstractFloat}
    # data
    y :: Vector{T}
    X :: Matrix{T}
    Z :: Matrix{T}
    # working arrays
    # whatever intermediate arrays you may want to pre-allocate
    storage_n :: Vector{T}
    storage_qq :: Matrix{T}
    storage_p  :: Vector{T}
    storage_q  :: Vector{T}
    ztz :: Matrix{T}
    yty :: T
    xty :: Vector{T}
    xtx :: Matrix{T}
    ztx :: Matrix{T}
    zty :: Vector{T}
    #last :: Vector{T}
end

# constructor
function LmmObs(
        y::Vector{T}, 
        X::Matrix{T}, 
        Z::Matrix{T}) where T <: AbstractFloat
    storage_n = similar(y)
    ztz = transpose(Z) * Z
    storage_qq = similar(ztz)
    storage_p  = Vector{T}(undef, size(X, 2))
    storage_q  = Vector{T}(undef, size(Z, 2))
    yty = transpose(y) * y
    xty = transpose(X) * y
    xtx = transpose(X) * X
    ztx = transpose(Z) * X
    zty = transpose(Z) * y
    #last = Vector{T}(undef, size(Z, 2))
    
    LmmObs(y, X, Z, storage_n, storage_qq, storage_p, storage_q, ztz, yty, xty, xtx, ztx, zty)
end

LmmObs

Write a function, with interface:

`logl!(obs, β, L, σ²)`

that evaluates the log-likelihood of the  $i$ -th datum. Here `L` is the lower triangular Cholesky factor from the Cholesky decomposition $\Sigma$=LL'. Make your code efficient in the $n_i >> q$  case. Think the intensive longitudinal measurement setting.

In [3]:
function logl!(
        obs :: LmmObs{T}, 
        β   :: Vector{T}, 
        L   :: Matrix{T}, 
        σ²  :: T) where T <: AbstractFloat
    n, p, q = size(obs.X, 1), size(obs.X, 2), size(obs.Z, 2)    
    # TODO: compute and return the log-likelihood
    
    #obs.storage_qq = 0
    #sleep(1e-3) # wait 1 ms as if your code takes 1ms
    
    aat = Matrix(I, q, q)  + 1/σ² * transpose(L) * transpose(Z) * Z * L
    chol = cholesky!(Symmetric(aat))
    
    return -n//2 * log(2π) - n//2 * log(σ²) - 1//2 * logdet(chol) -
    1/(2*(σ²)) * (obs.yty - 2*transpose(β)*obs.xty +  transpose(β)*obs.xtx*β) +
    1/(2*(σ²)^2) * (transpose((y - X*β)) * Z * L * inv(chol) * transpose(L) * transpose(Z) * (y- X*β))
    

end

logl! (generic function with 1 method)

#### Q3: Correctness

Compare your result (both accuracy and timing) to the Distributions.jl package using following data.

In [4]:
using BenchmarkTools, Distributions, LinearAlgebra, Random

Random.seed!(257)
# dimension
n, p, q = 2000, 5, 3
# predictors
X  = [ones(n) randn(n, p - 1)]
Z  = [ones(n) randn(n, q - 1)]
# parameter values
β  = [2.0; -1.0; rand(p - 2)]
σ² = 1.5
Σ  = fill(0.1, q, q) + 0.9I
# generate y
y  = X * β + Z * rand(MvNormal(Σ)) + sqrt(σ²) * randn(n)

# form an LmmObs object
obs = LmmObs(y, X, Z)

LmmObs{Float64}([-1.450910909560209, 1.5185224894450862, 5.265021705624027, 4.485272594164557, 0.6949699666429332, 1.7723256696372407, 1.1065838446466518, 3.7291668118296073, 4.288899999400642, 2.8241842645202406  …  4.058027151891635, 1.0909724390970443, 0.026692243086209766, -0.8927757653299448, 6.94725248926293, 3.519302085567343, 4.914007299083773, 2.1610206566690797, 1.857389542694909, 6.513818951020866], [1.0 0.6790633442371218 … 0.5400611947971554 -0.632040682052606; 1.0 1.2456776800889142 … -0.4818455756130373 0.6467830314674976; … ; 1.0 0.0733124748775436 … 0.6125080259511859 0.4181258283983667; 1.0 -1.336609049786048 … -0.18567490803712938 1.0745977099307227], [1.0 -1.0193326822839996 -0.15855601254314888; 1.0 1.7462667837699666 -0.4584376230657152; … ; 1.0 1.4843185594903878 0.42458303115266854; 1.0 0.3791714762820068 0.25150666970865837], [NaN, 6.9442117998086e-310, NaN, 6.9442117998608e-310, NaN, NaN, NaN, 6.94421179976277e-310, 6.94421390737607e-310, 6.94421179975487e-310

This is the standard way to evaluate log-density of a multivariate normal, using the Distributions.jl package. Let's evaluate the log-likelihood of this datum.

In [5]:
μ  = X * β
Ω  = Z * Σ * transpose(Z) +  σ² * I
mvn = MvNormal(μ, Symmetric(Ω)) # MVN(μ, Σ)
logpdf(mvn, y)

-3256.179335805832

Check that your answer matches that from Distributions.jl

In [6]:
L = Matrix(cholesky(Σ).L)
logl!(obs, β, L, σ²)

-3256.1793358058285

You will lose all 15 + 30 + 30 = 75 points if the following statement throws `AssertionError.`

In [14]:
@assert logl!(obs, β, Matrix(cholesky(Σ).L), σ²) ≈ logpdf(mvn, y)